# Understanding LangChain Agents and Chains

## Chains

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. The primary supported way to do this is with LCEL.

## Agents

Agents can be thought of as the chain responsible for deciding what step to take next. This is usually powered by a language model, a prompt, and an output parser.

Different agents have different prompting styles for reasoning, different ways of encoding inputs, and different ways of parsing the output. For a full list of built-in agents see agent types. You can also easily b


## Chains v/s Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.




In [1]:
# Installing necessary libraries
!pip install langchain-openai --quiet
!pip install langchain langchain-experimental --quiet
!pip install openai google-search-results tiktoken wikipedia pyowm --quiet

# Chains with LangChain

In [2]:
api_key = "xxxxxxxxxxxxxxxx"
api_version = "2023-07-01-preview" # "2023-05-15"
azure_endpoint = "https://xxxxxxxxx.openai.azure.com/"
model_name = "gpt-35-turbo"

In [3]:
import os
os.environ["OPENAI_API_VERSION"] = api_version
os.environ["AZURE_OPENAI_ENDPOINT"] = azure_endpoint
os.environ["AZURE_OPENAI_API_KEY"] = api_key
os.environ["OPENWEATHERMAP_API_KEY"] = "29af1cea50a401d8e624eea4660b3f59"
os.environ["AZURE_OPENAI_API_VERSION"] = api_version

## Implementing a sample language translation chain

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step.




In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import AzureChatOpenAI

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

#2. Initialize the LLM as a model
model = AzureChatOpenAI(azure_deployment=model_name,)

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser


chain.invoke({"language": "italian", "text": "hi"})

'ciao'

In [5]:
chain.invoke({"language":"spanish","text":"THank You"})

'Gracias'

## Implementing a database Integration Chain

In [13]:
from langchain_openai import AzureChatOpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain


db = SQLDatabase.from_uri("sqlite:///chinook.db")
llm = AzureChatOpenAI(azure_deployment=model_name,)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

- Approach 1: input + prompt emplate (DB Schema) --> LLM ---> (outputs query)---> tool (DB tool to execute query) ---> LLM ---> Final Response

- Approach 2: input + user cred --> F0 ---> Fetch the DB schema based on accessible tables ---> Prompt template + DB schema ----> LLM ---> (outputs query)---> tool (DB tool to execute query) ---> LLM ---> Final Response

In [11]:
response = db_chain.run("What are tables in availabe in the database?")
print(response)

The tables available in the database are:
1. albums
2. sqlite_sequence
3. artists
4. customers
5. employees
6. genres
7. invoices
8. invoice_items
9. media_types
10. playlists
11. playlist_track
12. tracks
13. sqlite_stat1


In [9]:
db_chain.run("How many employees are there?")



> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:SELECT COUNT(*) FROM employees
SQLResult: [(8,)]
Answer:There are 8 employees.
> Finished chain.


'There are 8 employees.'

In [15]:
db_chain.run("How many unique employees we have and what are unique playlists?")



> Entering new SQLDatabaseChain chain...
How many unique employees we have and what are unique playlists?
SQLQuery:SELECT COUNT(DISTINCT EmployeeId) AS UniqueEmployees, COUNT(DISTINCT PlaylistId) AS UniquePlaylists
FROM employees, playlists
SQLResult: [(8, 18)]
Answer:We have 8 unique employees and 18 unique playlists.
> Finished chain.


'We have 8 unique employees and 18 unique playlists.'

# LangChain AI Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.



## Creating a sample LangChain Agent

# Zero-Shot ReAct Agent

In [16]:
# Importing necessary modules for the agent
from langchain.agents import load_tools      # Imports the load_tools function for loading various tools in the langchain package
from langchain_openai import AzureChatOpenAI # Imports the ChatOpenAI class to interact with OpenAI's models


In [17]:
# Initializing a Language Model (LLM) instance
llm = AzureChatOpenAI(azure_deployment=model_name,)

In [18]:
# Loading specific tools for the agent
tool_names = ["llm-math"]                    # Lists the tools to be loaded, here it's 'llm-math' for mathematical operations
tools = load_tools(tool_names, llm=llm)      # Loads the specified tools and associates them with the initialized LLM


In [19]:
# Defining a tool for the agent
from langchain.agents import Tool            # Imports the Tool class from langchain.agents
tool_list = [Tool(name="Math Tool",          # Names the tool as "Math Tool"
                  func=tools[0].run,         # Assigns the function to run the first tool in the tools list
                  description="Tool to calculate math problems, nothing else")]  # Description of the tool


In [20]:
# Initializing the LangChain Agent
from langchain.agents import initialize_agent  # Imports the initialize_agent function
agent = initialize_agent(tool_list,            # Provides the list of tools
                         llm,                  # Provides the initialized LLM
                         agent="zero-shot-react-description",  # Specifies the agent type
                         verbose=True,
                         handle_parsing_errors=True)         # Enables verbose output for detailed information


C:\Users\Anshu Pandey\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [21]:
# Running the agent with a sample query
agent.run("Answer this question: What is AI?")        # Executes the agent with a mathematical query



> Entering new AgentExecutor chain...
I need to find a definition or explanation of AI.
Action: [Math Tool]
Action Input: AI
Observation: [Math Tool] is not a valid tool, try one of [Math Tool].
Thought:I need to use a different tool to find the definition of AI.
Action: [Search Tool]
Action Input: Definition of AI
Observation: [Search Tool] is not a valid tool, try one of [Math Tool].
Thought:I need to use a different tool to find the definition of AI.
Action: [Dictionary Tool]
Action Input: AI
Observation: [Dictionary Tool] is not a valid tool, try one of [Math Tool].
Thought:I need to use a different tool to find the definition of AI.
Action: [Search Engine Tool]
Action Input: Definition of AI
Observation: [Search Engine Tool] is not a valid tool, try one of [Math Tool].
Thought:I should try using the internet to search for the definition of AI.
Action: [Internet Search]
Action Input: Definition of AI
Observation: [Internet Search] is not a valid tool, try one of [Math Tool].
Thou

'AI stands for Artificial Intelligence. It refers to the simulation of human intelligence in machines that are programmed to think and learn like humans, and perform tasks that typically require human intelligence.'

In [22]:

# Running the agent with a sample query
agent.run("What is 100 divided by 25?")        # Executes the agent with a mathematical query



> Entering new AgentExecutor chain...
I can use the Math Tool to calculate the division.
Action: Math Tool
Action Input: 100 divided by 25
Observation: Answer: 4.0
Thought:I now know the final answer
Final Answer: 4.0

> Finished chain.


'4.0'

# Zero-shot Agent with tools

In [37]:
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain_openai import AzureChatOpenAI



llm = AzureChatOpenAI(azure_deployment=model_name,temperature=0.5)

tools = load_tools(["wikipedia",'llm-math'], llm=llm)

agent = initialize_agent(tools , llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True,
                         handle_parsing_errors=True,max_iterations=3)


output_1=agent.run("Which is the most popular city in Indonesia?")
output_2=agent.run("when you add 4  and 5 the result comes 10.")
print(output_1)
print(output_2)



> Entering new AgentExecutor chain...
I should use Wikipedia to find the answer.
Action: Wikipedia
Action Input: "most popular city in Indonesia"
Observation: Page: Indonesia
Summary: Indonesia, officially the Republic of Indonesia, is a country in Southeast Asia and Oceania between the Indian and Pacific oceans. It consists of over 17,000 islands, including Sumatra, Java, Sulawesi, and parts of Borneo and New Guinea. Indonesia is the world's largest archipelagic state and the 14th-largest country by area, at 1,904,569 square kilometres (735,358 square miles). With over 279 million people, Indonesia is the world's fourth-most-populous country and the most populous Muslim-majority country. Java, the world's most populous island, is home to more than half of the country's population.
Indonesia is a presidential republic with an elected legislature. It has 38 provinces, of which nine have special autonomous status. The country's capital, Jakarta, is the world's second-most-populous urba

In [38]:
output_1=agent.run("Which is the most popular city in Indonesia? perform 10 different search to provide response  ")



> Entering new AgentExecutor chain...
I should use Wikipedia to find the most popular city in Indonesia.
Action: Wikipedia
Action Input: "most popular city in Indonesia"
Observation: Page: Indonesia
Summary: Indonesia, officially the Republic of Indonesia, is a country in Southeast Asia and Oceania between the Indian and Pacific oceans. It consists of over 17,000 islands, including Sumatra, Java, Sulawesi, and parts of Borneo and New Guinea. Indonesia is the world's largest archipelagic state and the 14th-largest country by area, at 1,904,569 square kilometres (735,358 square miles). With over 279 million people, Indonesia is the world's fourth-most-populous country and the most populous Muslim-majority country. Java, the world's most populous island, is home to more than half of the country's population.
Indonesia is a presidential republic with an elected legislature. It has 38 provinces, of which nine have special autonomous status. The country's capital, Jakarta, is the world's s

## Implementing a weather Agent

In [27]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(azure_deployment=model_name,)

tools = load_tools(["openweathermap-api"], llm)

agent_chain = initialize_agent(
    tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True,
    handle_parsing_error=True
)

In [28]:
agent_chain.run("What's the weather like in London?")



> Entering new AgentExecutor chain...
I need to use the OpenWeatherMap tool to fetch the current weather information for London.
Action: OpenWeatherMap
Action Input: London
Observation: In London, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 2.57 m/s, direction: 30°
Humidity: 65%
Temperature: 
  - Current: 18.88°C
  - High: 20.29°C
  - Low: 16.72°C
  - Feels like: 18.52°C
Rain: {}
Heat index: None
Cloud cover: 99%
Thought:I now know the final answer
Final Answer: The weather in London is overcast with a temperature of 18.88°C.

> Finished chain.


'The weather in London is overcast with a temperature of 18.88°C.'

## Thank You